In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning - Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Redução de Dimensionalidade
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline
pd.set_option('display.max_columns', None)

# Seed para reprodutibilidade
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 3.1 Carregamento dos Dados

In [ ]:
# Carregar dados
print("Carregando dados...")
flights = pd.read_csv('flights.csv')
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')

print(f"Flights: {flights.shape}")
print(f"Airlines: {airlines.shape}")
print(f"Airports: {airports.shape}")

## 3.2 Clusterização de Aeroportos

Vamos agrupar aeroportos com base em características operacionais:
- Volume de voos
- Taxa de atraso
- Atraso médio
- Distância média das rotas

In [ ]:
# Criar features por aeroporto de origem
print("Criando features de aeroportos...")

# Agregar informações por aeroporto
airport_features = flights.groupby('ORIGIN_AIRPORT').agg({
    'FLIGHT_NUMBER': 'count',  # Total de voos
    'DISTANCE': 'mean',  # Distância média
    'ARRIVAL_DELAY': ['mean', 'median', 'std'],  # Estatísticas de atraso
    'DEPARTURE_DELAY': ['mean', 'median'],
    'CANCELLED': 'sum',  # Cancelamentos
    'DIVERTED': 'sum'  # Desvios
}).reset_index()

# Renomear colunas
airport_features.columns = ['AIRPORT', 'TOTAL_FLIGHTS', 'AVG_DISTANCE', 
                            'AVG_ARRIVAL_DELAY', 'MEDIAN_ARRIVAL_DELAY', 'STD_ARRIVAL_DELAY',
                            'AVG_DEPARTURE_DELAY', 'MEDIAN_DEPARTURE_DELAY',
                            'TOTAL_CANCELLED', 'TOTAL_DIVERTED']

# Calcular taxas
airport_features['CANCELLATION_RATE'] = (airport_features['TOTAL_CANCELLED'] / 
                                          airport_features['TOTAL_FLIGHTS']) * 100
airport_features['DIVERSION_RATE'] = (airport_features['TOTAL_DIVERTED'] / 
                                       airport_features['TOTAL_FLIGHTS']) * 100

# Taxa de atraso (>15 minutos)
delayed_counts = flights[flights['ARRIVAL_DELAY'] > 15].groupby('ORIGIN_AIRPORT').size()
airport_features['DELAY_RATE'] = (delayed_counts / airport_features['TOTAL_FLIGHTS']) * 100
airport_features['DELAY_RATE'].fillna(0, inplace=True)

# Merge com informações dos aeroportos
airport_features = airport_features.merge(airports, left_on='AIRPORT', 
                                          right_on='IATA_CODE', how='left')

print(f"\nShape: {airport_features.shape}")
print(f"\nPrimeiras linhas:")
print(airport_features.head())

In [ ]:
# Visualizar distribuições
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

features_to_plot = ['TOTAL_FLIGHTS', 'AVG_DISTANCE', 'AVG_ARRIVAL_DELAY', 
                    'DELAY_RATE', 'CANCELLATION_RATE', 'DIVERSION_RATE']

for idx, feature in enumerate(features_to_plot):
    row = idx // 3
    col = idx % 3
    
    airport_features[feature].hist(bins=50, ax=axes[row, col], edgecolor='black')
    axes[row, col].set_title(f'Distribuição de {feature}', fontweight='bold')
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel('Frequência')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Preparar dados para clustering
# Selecionar features relevantes e remover NaN
clustering_features = ['TOTAL_FLIGHTS', 'AVG_DISTANCE', 'AVG_ARRIVAL_DELAY', 
                       'DELAY_RATE', 'CANCELLATION_RATE', 'STD_ARRIVAL_DELAY']

# Filtrar aeroportos com pelo menos 100 voos
airport_cluster_data = airport_features[airport_features['TOTAL_FLIGHTS'] >= 100].copy()
airport_cluster_data = airport_cluster_data[clustering_features + ['AIRPORT', 'AIRPORT_NAME']].dropna()

X_airports = airport_cluster_data[clustering_features].values

print(f"Número de aeroportos para clustering: {len(X_airports)}")
print(f"Shape dos dados: {X_airports.shape}")

# Normalização
scaler_airports = StandardScaler()
X_airports_scaled = scaler_airports.fit_transform(X_airports)

print("✓ Dados normalizados")

In [ ]:
# Determinar número ótimo de clusters usando Elbow Method e Silhouette Score
print("Determinando número ótimo de clusters...")

inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_airports_scaled)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_airports_scaled, kmeans.labels_))
    
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores[-1]:.3f}")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Elbow Method
axes[0].plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Número de Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inércia', fontsize=12)
axes[0].set_title('Método do Cotovelo (Elbow Method)', fontweight='bold', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Silhouette Score
axes[1].plot(k_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Número de Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score por k', fontweight='bold', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Melhor k baseado em Silhouette
best_k = k_range[np.argmax(silhouette_scores)]
print(f"\nMelhor k (Silhouette Score): {best_k}")

In [ ]:
# Aplicar K-Means com o melhor k
optimal_k = best_k  # Ou escolher manualmente, ex: 4

print(f"\nAplicando K-Means com k={optimal_k}...")
kmeans = KMeans(n_clusters=optimal_k, random_state=RANDOM_STATE, n_init=10)
airport_cluster_data['CLUSTER'] = kmeans.fit_predict(X_airports_scaled)

print(f"\nDistribuição de aeroportos por cluster:")
print(airport_cluster_data['CLUSTER'].value_counts().sort_index())

# Métricas de qualidade
silhouette_avg = silhouette_score(X_airports_scaled, airport_cluster_data['CLUSTER'])
calinski_score = calinski_harabasz_score(X_airports_scaled, airport_cluster_data['CLUSTER'])
davies_bouldin = davies_bouldin_score(X_airports_scaled, airport_cluster_data['CLUSTER'])

print(f"\nMétricas de Qualidade do Clustering:")
print(f"Silhouette Score: {silhouette_avg:.3f} (quanto maior, melhor)")
print(f"Calinski-Harabasz Score: {calinski_score:.2f} (quanto maior, melhor)")
print(f"Davies-Bouldin Score: {davies_bouldin:.3f} (quanto menor, melhor)")

In [ ]:
# Analisar características de cada cluster
print("\n" + "="*80)
print("ANÁLISE DOS CLUSTERS DE AEROPORTOS")
print("="*80)

cluster_analysis = airport_cluster_data.groupby('CLUSTER')[clustering_features].mean()
print("\nMédia das características por cluster:")
print(cluster_analysis.round(2))

# Visualização
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, feature in enumerate(clustering_features):
    row = idx // 3
    col = idx % 3
    
    cluster_analysis[feature].plot(kind='bar', ax=axes[row, col])
    axes[row, col].set_title(f'{feature} por Cluster', fontweight='bold')
    axes[row, col].set_xlabel('Cluster')
    axes[row, col].set_ylabel(feature)
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].set_xticklabels(axes[row, col].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Nomear clusters baseado nas características
print("\nExemplos de aeroportos por cluster:")
print("="*80)

for cluster in range(optimal_k):
    print(f"\nCLUSTER {cluster}:")
    cluster_airports = airport_cluster_data[airport_cluster_data['CLUSTER'] == cluster]
    
    # Top 10 aeroportos por volume
    top_airports = cluster_airports.nlargest(10, 'TOTAL_FLIGHTS')[['AIRPORT', 'AIRPORT_NAME', 
                                                                     'TOTAL_FLIGHTS', 'AVG_ARRIVAL_DELAY', 
                                                                     'DELAY_RATE']]
    print(top_airports.to_string(index=False))
    
    # Características médias
    print(f"\nCaracterísticas médias do cluster:")
    print(f"  Total de voos: {cluster_airports['TOTAL_FLIGHTS'].mean():.0f}")
    print(f"  Distância média: {cluster_airports['AVG_DISTANCE'].mean():.0f} milhas")
    print(f"  Atraso médio: {cluster_airports['AVG_ARRIVAL_DELAY'].mean():.1f} min")
    print(f"  Taxa de atraso: {cluster_airports['DELAY_RATE'].mean():.1f}%")

## 3.3 Redução de Dimensionalidade com PCA

In [ ]:
# Aplicar PCA
print("Aplicando PCA...")

pca = PCA()
X_pca = pca.fit_transform(X_airports_scaled)

# Variância explicada
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"\nVariância explicada por componente:")
for i, (var, cum_var) in enumerate(zip(explained_variance, cumulative_variance)):
    print(f"PC{i+1}: {var*100:.2f}% (Cumulativa: {cum_var*100:.2f}%)")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Scree plot
axes[0].bar(range(1, len(explained_variance)+1), explained_variance*100)
axes[0].set_xlabel('Componente Principal', fontsize=12)
axes[0].set_ylabel('Variância Explicada (%)', fontsize=12)
axes[0].set_title('Scree Plot - Variância Explicada por PC', fontweight='bold', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Variância cumulativa
axes[1].plot(range(1, len(cumulative_variance)+1), cumulative_variance*100, 'ro-', linewidth=2)
axes[1].axhline(y=80, color='g', linestyle='--', label='80% Variância')
axes[1].axhline(y=90, color='b', linestyle='--', label='90% Variância')
axes[1].set_xlabel('Número de Componentes', fontsize=12)
axes[1].set_ylabel('Variância Cumulativa (%)', fontsize=12)
axes[1].set_title('Variância Cumulativa Explicada', fontweight='bold', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Loadings - Contribuição de cada feature nos PCs
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(len(pca.components_))],
    index=clustering_features
)

print("\nLoadings (Contribuição das Features):")
print(loadings.round(3))

# Visualizar loadings dos 2 primeiros PCs
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

loadings['PC1'].plot(kind='barh', ax=axes[0])
axes[0].set_title('Loadings - PC1', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Loading')
axes[0].grid(True, alpha=0.3)

loadings['PC2'].plot(kind='barh', ax=axes[1], color='orange')
axes[1].set_title('Loadings - PC2', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Loading')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualização 2D dos clusters usando as 2 primeiras componentes principais
airport_cluster_data['PC1'] = X_pca[:, 0]
airport_cluster_data['PC2'] = X_pca[:, 1]

plt.figure(figsize=(14, 10))

for cluster in range(optimal_k):
    cluster_data = airport_cluster_data[airport_cluster_data['CLUSTER'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], 
               label=f'Cluster {cluster}', s=100, alpha=0.6)

# Adicionar nomes dos principais aeroportos
top_airports = airport_cluster_data.nlargest(20, 'TOTAL_FLIGHTS')
for idx, row in top_airports.iterrows():
    plt.annotate(row['AIRPORT'], (row['PC1'], row['PC2']), 
                fontsize=9, alpha=0.7)

plt.xlabel(f'PC1 ({explained_variance[0]*100:.1f}% variância)', fontsize=12)
plt.ylabel(f'PC2 ({explained_variance[1]*100:.1f}% variância)', fontsize=12)
plt.title('Clusters de Aeroportos - Projeção PCA (2D)', fontweight='bold', fontsize=16)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.4 Clusterização de Companhias Aéreas

In [ ]:
# Criar features por companhia aérea
print("Criando features de companhias aéreas...")

airline_features = flights.groupby('AIRLINE').agg({
    'FLIGHT_NUMBER': 'count',
    'DISTANCE': 'mean',
    'ARRIVAL_DELAY': ['mean', 'median', 'std'],
    'DEPARTURE_DELAY': ['mean', 'median'],
    'CANCELLED': 'sum',
    'DIVERTED': 'sum'
}).reset_index()

airline_features.columns = ['AIRLINE_CODE', 'TOTAL_FLIGHTS', 'AVG_DISTANCE',
                            'AVG_ARRIVAL_DELAY', 'MEDIAN_ARRIVAL_DELAY', 'STD_ARRIVAL_DELAY',
                            'AVG_DEPARTURE_DELAY', 'MEDIAN_DEPARTURE_DELAY',
                            'TOTAL_CANCELLED', 'TOTAL_DIVERTED']

# Taxas
airline_features['CANCELLATION_RATE'] = (airline_features['TOTAL_CANCELLED'] / 
                                         airline_features['TOTAL_FLIGHTS']) * 100
airline_features['DIVERSION_RATE'] = (airline_features['TOTAL_DIVERTED'] / 
                                      airline_features['TOTAL_FLIGHTS']) * 100

# Taxa de atraso
delayed_airlines = flights[flights['ARRIVAL_DELAY'] > 15].groupby('AIRLINE').size()
airline_features['DELAY_RATE'] = (delayed_airlines / airline_features['TOTAL_FLIGHTS']) * 100
airline_features['DELAY_RATE'].fillna(0, inplace=True)

# Merge com nomes
airline_features = airline_features.merge(airlines, left_on='AIRLINE_CODE', 
                                         right_on='IATA_CODE', how='left')

print(f"\nShape: {airline_features.shape}")
print(airline_features[['AIRLINE_CODE', 'AIRLINE', 'TOTAL_FLIGHTS', 'AVG_ARRIVAL_DELAY', 'DELAY_RATE']])

In [ ]:
# Preparar dados para clustering de companhias
airline_clustering_features = ['TOTAL_FLIGHTS', 'AVG_DISTANCE', 'AVG_ARRIVAL_DELAY',
                               'DELAY_RATE', 'CANCELLATION_RATE', 'STD_ARRIVAL_DELAY']

X_airlines = airline_features[airline_clustering_features].fillna(0).values

# Normalização
scaler_airlines = StandardScaler()
X_airlines_scaled = scaler_airlines.fit_transform(X_airlines)

print(f"Número de companhias: {len(X_airlines)}")
print("✓ Dados normalizados")

In [ ]:
# Aplicar K-Means para companhias (k=3 ou 4)
k_airlines = 3

print(f"Aplicando K-Means com k={k_airlines}...")
kmeans_airlines = KMeans(n_clusters=k_airlines, random_state=RANDOM_STATE, n_init=10)
airline_features['CLUSTER'] = kmeans_airlines.fit_predict(X_airlines_scaled)

print(f"\nDistribuição por cluster:")
print(airline_features['CLUSTER'].value_counts().sort_index())

# Análise dos clusters
print("\n" + "="*80)
print("CLUSTERS DE COMPANHIAS AÉREAS")
print("="*80)

for cluster in range(k_airlines):
    print(f"\nCLUSTER {cluster}:")
    cluster_airlines = airline_features[airline_features['CLUSTER'] == cluster]
    print(cluster_airlines[['AIRLINE_CODE', 'AIRLINE', 'TOTAL_FLIGHTS', 
                            'AVG_ARRIVAL_DELAY', 'DELAY_RATE']].to_string(index=False))
    
    print(f"\nCaracterísticas médias:")
    print(f"  Voos: {cluster_airlines['TOTAL_FLIGHTS'].mean():.0f}")
    print(f"  Atraso médio: {cluster_airlines['AVG_ARRIVAL_DELAY'].mean():.1f} min")
    print(f"  Taxa de atraso: {cluster_airlines['DELAY_RATE'].mean():.1f}%")
    print(f"  Taxa de cancelamento: {cluster_airlines['CANCELLATION_RATE'].mean():.2f}%")

In [ ]:
# Visualização dos clusters de companhias
pca_airlines = PCA(n_components=2)
X_airlines_pca = pca_airlines.fit_transform(X_airlines_scaled)

airline_features['PC1'] = X_airlines_pca[:, 0]
airline_features['PC2'] = X_airlines_pca[:, 1]

plt.figure(figsize=(12, 8))

for cluster in range(k_airlines):
    cluster_data = airline_features[airline_features['CLUSTER'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'],
               label=f'Cluster {cluster}', s=200, alpha=0.7)

# Adicionar nomes
for idx, row in airline_features.iterrows():
    plt.annotate(row['AIRLINE_CODE'], (row['PC1'], row['PC2']),
                fontsize=11, fontweight='bold', alpha=0.8)

plt.xlabel(f'PC1 ({pca_airlines.explained_variance_ratio_[0]*100:.1f}% variância)', fontsize=12)
plt.ylabel(f'PC2 ({pca_airlines.explained_variance_ratio_[1]*100:.1f}% variância)', fontsize=12)
plt.title('Clusters de Companhias Aéreas - Projeção PCA', fontweight='bold', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.5 Análise de Rotas (Clustering)

In [ ]:
# Criar features por rota (origem-destino)
print("Criando features de rotas...")

flights['ROUTE'] = flights['ORIGIN_AIRPORT'] + '-' + flights['DESTINATION_AIRPORT']

route_features = flights.groupby('ROUTE').agg({
    'FLIGHT_NUMBER': 'count',
    'DISTANCE': 'mean',
    'ARRIVAL_DELAY': ['mean', 'std'],
    'CANCELLED': 'sum'
}).reset_index()

route_features.columns = ['ROUTE', 'TOTAL_FLIGHTS', 'DISTANCE', 
                          'AVG_DELAY', 'STD_DELAY', 'TOTAL_CANCELLED']

route_features['CANCELLATION_RATE'] = (route_features['TOTAL_CANCELLED'] / 
                                       route_features['TOTAL_FLIGHTS']) * 100

# Filtrar rotas com pelo menos 50 voos
route_features = route_features[route_features['TOTAL_FLIGHTS'] >= 50]

print(f"\nNúmero de rotas: {len(route_features)}")
print(f"\nTop 10 rotas por volume:")
print(route_features.nlargest(10, 'TOTAL_FLIGHTS')[['ROUTE', 'TOTAL_FLIGHTS', 'AVG_DELAY']])

In [ ]:
# Clustering de rotas (amostra para performance)
if len(route_features) > 500:
    route_sample = route_features.nlargest(500, 'TOTAL_FLIGHTS')
else:
    route_sample = route_features.copy()

route_clustering_features = ['TOTAL_FLIGHTS', 'DISTANCE', 'AVG_DELAY', 'STD_DELAY', 'CANCELLATION_RATE']
X_routes = route_sample[route_clustering_features].fillna(0).values

scaler_routes = StandardScaler()
X_routes_scaled = scaler_routes.fit_transform(X_routes)

# K-Means
k_routes = 4
kmeans_routes = KMeans(n_clusters=k_routes, random_state=RANDOM_STATE, n_init=10)
route_sample['CLUSTER'] = kmeans_routes.fit_predict(X_routes_scaled)

print(f"\nClusters de rotas (k={k_routes}):")
print(route_sample['CLUSTER'].value_counts().sort_index())

# Análise
print("\n" + "="*80)
print("CLUSTERS DE ROTAS")
print("="*80)

for cluster in range(k_routes):
    print(f"\nCLUSTER {cluster}:")
    cluster_routes = route_sample[route_sample['CLUSTER'] == cluster]
    
    print(f"Número de rotas: {len(cluster_routes)}")
    print(f"Características médias:")
    print(f"  Voos por rota: {cluster_routes['TOTAL_FLIGHTS'].mean():.0f}")
    print(f"  Distância média: {cluster_routes['DISTANCE'].mean():.0f} milhas")
    print(f"  Atraso médio: {cluster_routes['AVG_DELAY'].mean():.1f} min")
    print(f"  Taxa de cancelamento: {cluster_routes['CANCELLATION_RATE'].mean():.2f}%")
    
    print(f"\nTop 5 rotas do cluster:")
    top_5 = cluster_routes.nlargest(5, 'TOTAL_FLIGHTS')[['ROUTE', 'TOTAL_FLIGHTS', 
                                                          'DISTANCE', 'AVG_DELAY']]
    print(top_5.to_string(index=False))

## 3.6 Visualização Interativa com Plotly

In [ ]:
# Scatter plot interativo dos aeroportos
fig = px.scatter(
    airport_cluster_data,
    x='PC1',
    y='PC2',
    color='CLUSTER',
    size='TOTAL_FLIGHTS',
    hover_data=['AIRPORT', 'AIRPORT_NAME', 'TOTAL_FLIGHTS', 'AVG_ARRIVAL_DELAY', 'DELAY_RATE'],
    title='Clusters de Aeroportos - Visualização Interativa',
    labels={'PC1': f'PC1 ({explained_variance[0]*100:.1f}%)',
            'PC2': f'PC2 ({explained_variance[1]*100:.1f}%)',
            'CLUSTER': 'Cluster'},
    width=1000,
    height=700
)

fig.update_traces(marker=dict(line=dict(width=1, color='white')))
fig.update_layout(font=dict(size=12))
fig.show()

## 3.7 Conclusões da Modelagem Não Supervisionada

### Clusterização de Aeroportos:
- Identificamos diferentes perfis de aeroportos baseados em:
  - Volume de operações
  - Padrões de atraso
  - Características das rotas
- Clusters revelam categorias como: hubs principais, aeroportos regionais, aeroportos problemáticos

### Clusterização de Companhias Aéreas:
- Agrupamento baseado em performance operacional
- Identificação de companhias com padrões similares de pontualidade
- Clusters revelam: grandes operadoras, companhias regionais, low-cost carriers

### Clusterização de Rotas:
- Rotas agrupadas por características similares
- Identificação de rotas problemáticas vs rotas eficientes
- Padrões relacionados a distância e frequência

### PCA (Redução de Dimensionalidade):
- Principais componentes explicam a maior parte da variância
- Permite visualização eficiente dos clusters
- Identifica quais features são mais importantes

### Insights:
1. Aeroportos podem ser categorizados em perfis distintos
2. Companhias aéreas mostram padrões operacionais específicos
3. Rotas têm características próprias independente da companhia
4. A redução de dimensionalidade facilita a interpretação

### Aplicações Práticas:
- Estratégias diferenciadas por tipo de aeroporto
- Benchmarking entre companhias do mesmo cluster
- Otimização de rotas problemáticas
- Alocação de recursos baseada em perfis

In [ ]:
# Salvar resultados
print("Salvando resultados...")

airport_cluster_data.to_csv('airport_clusters.csv', index=False)
airline_features.to_csv('airline_clusters.csv', index=False)
route_sample.to_csv('route_clusters.csv', index=False)

print("✓ Resultados salvos")
print("\n" + "="*80)
print("MODELAGEM NÃO SUPERVISIONADA CONCLUÍDA!")
print("="*80)